# DES 암호화 및 복호화

DES 암호 시스템의 구조를 살펴봅시다.

## 1. DES를 사용하기 위한 값들 정의
DES 암호화 및 복호화를 사용하기에 앞서, 우리는 몇 가지 행렬을 사용해야합니다. 예를 들면 초기전치 IP, 전치 확장 E 행렬이나, 순열 P 행렬과 같은 것들입니다. 

### 1.1. IP & IP-1
우리는 DES 암호화를 할 때 초기 전치함수와 초기 전치함수의 역함수가 있다고 배웠습니다.  
해당 값은 다음과 같이 이루어져 있었습니다.

In [ ]:
IP = [58, 50, 42, 34, 26, 18, 10, 2,
      60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6,
      64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1,
      59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5,
      63, 55, 47, 39, 31, 23, 15, 7]

IP_inv = [40, 8, 48, 16, 56, 24, 64, 32,
          39, 7, 47, 15, 55, 23, 63, 31,
          38, 6, 46, 14, 54, 22, 62, 30,
          37, 5, 45, 13, 53, 21, 61, 29,
          36, 4, 44, 12, 52, 20, 60, 28,
          35, 3, 43, 11, 51, 19, 59, 27,
          34, 2, 42, 10, 50, 18, 58, 26,
          33, 1, 41, 9, 49, 17, 57, 25]

### 1.2. 확장 전치함수 E
우리는 DES 암호화 중 f 함수에서 확장 치환 테이블 E에 대해 다루었습니다. 이는 다음과 같이 이루어져 있습니다.

In [ ]:
E = [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9,
     8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17,
     16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25,
     24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1]

### 1.3. S-box
우리는 DES 암호화 중 f 함수에서 S-box를 이용해 연산한다는 것을 봤습니다. 우리는 한 가지 S-box에 대해서만 봤지만, 각 8개의 그룹으로 나뉘기 때문에 8개의 S-box가 필요하며 이는 다음과 같습니다.

In [ ]:
S_boxes = [
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]],
]

### 1.4. 순열 함수 P
우리는 DES 암호화 중 f 함수에서 순열 테이블 P에 대해 다루었습니다. 이는 다음과 같이 이루어져 있습니다.

In [ ]:
P = [16, 7, 20, 21,
     29, 12, 28, 17,
     1, 15, 23, 26,
     5, 18, 31, 10,
     2, 8, 24, 14,
     32, 27, 3, 9,
     19, 13, 30, 6,
     22, 11, 4, 25]

### 1.5. 키 생성에 필요한 테이블 PC1, PC2
우리는 키 생성 테이블로 PC1, PC2를 사용했습니다. 이는 다음과 같이 이루어져 있습니다.

In [ ]:
PC1 = [57, 49, 41, 33, 25, 17, 9,
       1, 58, 50, 42, 34, 26, 18,
       10, 2, 59, 51, 43, 35, 27,
       19, 11, 3, 60, 52, 44, 36,
       63, 55, 47, 39, 31, 23, 15,
       7, 62, 54, 46, 38, 30, 22,
       14, 6, 61, 53, 45, 37, 29,
       21, 13, 5, 28, 20, 12, 4]

PC2 = [14, 17, 11, 24, 1, 5,
       3, 28, 15, 6, 21, 10,
       23, 19, 12, 4, 26, 8,
       16, 7, 27, 20, 13, 2,
       41, 52, 31, 37, 47, 55,
       30, 40, 51, 45, 33, 48,
       44, 49, 39, 56, 34, 53,
       46, 42, 50, 36, 29, 32]

### 1.6. 키 생성에 필요한 테이블 LS
우리는 키 생성을 위해 left shift 연산을 수행했습니다. 이는 다음과 같이 이루어져 있습니다.

In [ ]:
shifts = [1, 1, 2, 2, 2, 2, 2, 2,
          1, 2, 2, 2, 2, 2, 2, 1]

## 2. DES를 사용하기 위한 함수들 정의

우리는 행렬의 위치를 바꾸는 전치함수를 만들어야 합니다. 이 전치함수는 permute 함수로 구현합시다.  
이 함수는 P 순열 연산 뿐만 아니라 E 확장 연산에도 아주 유용하게 쓰입니다.  
IP 초기치환과 IP-1 역치환도 사실 있는 위치를 바꿔주는 연산이었죠.  
게다가, Key를 생성할 때 PC1, PC2를 적용하는 과정도 전치과정이 필요했죠.

In [ ]:
def permute(block, table):
    return [block[x - 1] for x in table]

XOR 연산도 우린 구현해야 합니다. XOR 연산은 이미 python에서 ^ 로 정의되어있지만, 이걸 여러 값들을 한꺼번에 하기 위해 따로 정의해줍니다.

In [ ]:
def xor(t1, t2):
    return [x ^ y for x, y in zip(t1, t2)]

다음은 키 생성을 위해 Left shift 하는 함수입니다. 이는 위의 shifts에 기반한 값만큼 left shift 하게 해줍니다.

In [ ]:
def shift_left(block, n):
    return block[n:] + block[:n]

마지막으로 s-box를 적용해주는 함수입니다. 이 함수는 f 함수내의 s-box 연산을 하게 해줍니다.  
우리 row와 column은 각각 끝의 두 값과, 가운데 끼인 네 값으로 이루어져있다는 것을 확인했을 겁니다.  
이를 다음과 같이 구현해줍니다.  
i*6은 각 S는 6개 단위로 쪼개지기 때문에 이렇게 둔 것이며, 0번째 있는 값과 5번째 있는 값으로 row를 구성해주고, 1~4번째 값으로 column을 구성해줍니다.  

In [ ]:
def sbox_substitution(block):
    result = []
    for i in range(8):
        row = (block[i*6] << 1) + block[i*6 + 5]
        col = (block[i*6 + 1] << 3) + (block[i*6 + 2] << 2) + (block[i*6 + 3] << 1) + block[i*6 + 4]
        val = S_boxes[i][row][col]
        result += [val >> 3 & 1, val >> 2 & 1, val >> 1 & 1, val & 1]
    return result

## 3. DES 암호화 및 복호화 정의
우리는 DES 암호화 및 복호화를 python의 class로 정의하려 합니다. 

이때 뭐가 필요하냐면,  
(1) key를 생성해줘야합니다. (init에서 처음 key 생성)    
(2) 라운드마다 key를 연산해줘야합니다. (generate_sub_keys에서 작은 key 값 16개 연산, 이때 key의 길이를 맞추기 위해 064b로 만들어줌)     
(3) 암호화 과정 - IP - 32bit씩 쪼개어 f연산 - IP-1를 반복 연산해줘야 합니다. (encrypt에서 16회의 key와 연산)   
(4) 복호화 과정 -  IP - 32bit씩 쪼개어 f연산 - IP-1를 반복 연산해줘야 합니다. (decrypt에서 16회의 round 연산)   

In [ ]:
class DES:
    def __init__(self, key):
        self.key = key
        self.sub_keys = self.generate_sub_keys()

    def generate_sub_keys(self):
        key_bits = [int(x) for x in '{:064b}'.format(self.key)]   ## 여기서 64bit 앞의 0을 채워두었기 때문에, 자동으로 16개의 subkey가 생성됨!
        key_bits = permute(key_bits, PC1)
        C, D = key_bits[:28], key_bits[28:]
        sub_keys = []
        for shift in shifts:
            C = shift_left(C, shift)
            D = shift_left(D, shift)
            sub_keys.append(permute(C + D, PC2))
        return sub_keys

    def feistel(self, right, sub_key):
        right_expanded = permute(right, E)
        temp = xor(right_expanded, sub_key)
        temp = sbox_substitution(temp)
        temp = permute(temp, P)
        return temp

    def encrypt_block(self, block):
        block = permute(block, IP)
        L, R = block[:32], block[32:]
        for sub_key in self.sub_keys:  ## 여기서 subkeys의 개수만큼 수행하므로, 16번 반복수행함!
            L, R = R, xor(L, self.feistel(R, sub_key))
        cipher_block = permute(R + L, IP_inv)
        return cipher_block

    def decrypt_block(self, block):
        block = permute(block, IP)
        L, R = block[:32], block[32:]
        for sub_key in reversed(self.sub_keys):  ## 여기서 subkeys의 개수만큼 수행하므로, 16번 반복수행함!
            L, R = R, xor(L, self.feistel(R, sub_key))
        cipher_block = permute(R + L, IP_inv)
        return cipher_block

    def encrypt(self, plaintext):
        plaintext_bits = [int(x) for x in '{:064b}'.format(plaintext)]
        cipher_bits = self.encrypt_block(plaintext_bits)
        ciphertext = int(''.join(str(x) for x in cipher_bits), 2)
        return ciphertext

    def decrypt(self, ciphertext):
        ciphertext_bits = [int(x) for x in '{:064b}'.format(ciphertext)]
        plain_bits = self.decrypt_block(ciphertext_bits)
        plaintext = int(''.join(str(x) for x in plain_bits), 2)
        return plaintext

## 4. 만든 DES class 사용

이제 앞서 만들었던 DES 클래스를 사용해봅시다. 우선 DES에 key를 만들고 이 key로 암호화를 해봅시다.

예시 코드입니다. "123123" 라는 값으로 key를 만들었습니다.  

In [ ]:
key = 123123  # 64-bit key for DES
des1 = DES(key)

이 key를 이용해서 "1231232131"라는 문자열을 암호화해봅시다.

In [ ]:
des1.encrypt(1231232131)

이상한 값이 나왔죠? 이걸 복호화하면 원래 값이 나오는지 확인해봅시다.

In [ ]:
des1.decrypt(2941798411662220893)

## 5. 만든 DES class를 직접 사용해보기

이번에는, 여러분들이 직접 사용해봅시다. 위의 예시를 참고해서 key 값이 100001일때로 121211101000011 이라는 평문을 암호화해봅시다.  
또, 복호화도 해봅시다.